Regression in real state 
========================

A partir de um dataset com dados de imóveis (metragem, cidade, bairro, quartos, etc), queremos prever o valor de cada registro usando redes neurais.

O dataset foi obtido a partir de _webscrapping_ em sites de publicação de anúncios 

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math 

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping

import sys
import platform

sys.path.insert(1, '../')

from src.plot import plot_heatmap
from src.train import get_all_subsets, results_regression


C:\Users\psebastianv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Config

In [ ]:
# Evita mostrar números em formato científico (para melhor visualização)
np.set_printoptions(suppress=True)

# tamanhos padrão e visualização de plots
PLOT_WIDE = (14,7)
PLOT_MEDIUM = (10,8)
sns.set_style("darkgrid")

## Load data

In [ ]:
WIN = r'C:\Users\psebastianv\dev\python\projetos-e-testes\other_python\zap-project\results\imoveis_data.csv'
LNX = r'./../../../../../python/other_python/zap-project/results/imoveis_data.csv'
data = pd.read_csv(WIN if platform.platform().startswith("Windows") else LNX)

In [ ]:
data.shape

In [ ]:
data.head()

## Preprocessing

In [ ]:
# fields numbers
list(enumerate(data.columns))

In [ ]:
# Limpar colunas desnecessárias 
data.drop(data.columns[[0, 1, 5, 8, 9, 10, 11]], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
# missing values
data.isna().sum()

In [ ]:
# Duplicates
data.drop_duplicates(keep='first', inplace=True)
data.duplicated().sum()

In [ ]:
# nas características rooms, wc e garage_spots temos alguns valores não numéricos que indicam, por exemplo 2-3 \
# vamos rodar uma função para calcular a média desses valores e ter um resultado mais acurado 
data.rooms.unique()

In [ ]:
# Tratamento de valores numéricos que estão constando como str
# por exemplo: rooms nem sempre é número: 1 - 2

def update_data(i):
    output = None
    try:
        output = int(i)
    except:
        output = np.array(i.split(' - ')).astype(int).mean()
    
    return output

assert update_data('1') == 1, 'Não foi possível castear to int'
assert update_data('2 - 3') == 2.5, 'Não foi possível castear to int'


In [ ]:
# Atualizar 3 colunas
for col in list(data.iloc[:, :4].select_dtypes('object').columns):
    data[col] = data[col].map(lambda x : update_data(x)).astype(float)

data.dtypes, data.head()

In [ ]:
plot_heatmap(data.corr(numeric_only=True), \
             'Correlação de variáveis')

In [ ]:
data.head()

In [ ]:
# Aplicando one-hot encoding na coluna 'neighborhood'
data = pd.get_dummies(data, columns=['neighborhood'], prefix='neighborhood', drop_first=True)

In [ ]:
data.isna().sum().sum()

## Particionamento da base

In [ ]:
y = data['price']
X = data.drop('price', axis=1)

In [ ]:
# Normalizar
y /= np.max(y)
X /= np.max(X, axis=0)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_all_subsets(X, y)

In [ ]:
print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_val: ', y_val.shape)
print('y_test: ', y_test.shape)


## Treinamento, definição de arquitetura

In [ ]:
dropout_pct = 0.3
shape = (X_train.shape[1],)

display(shape)

model = Sequential()
model.add(Dense(units=512, activation='relu', input_shape=shape))
model.add(Dropout(dropout_pct))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(dropout_pct))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(dropout_pct))

model.add(Dense(units=1))

model.summary()

## Definição de otimizadores

In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0.001, \
                   patience=10, verbose=1, mode='auto')

historico = model.fit(X_train,
                      y_train,
                      epochs=1000,
                      batch_size=32,
                      verbose=1,
                      validation_data=(X_val, y_val),
                      callbacks=[es])


In [ ]:
y_pred = model.predict(X_test)

## Avaliação do modelo

In [ ]:
results_regression(y_test, y_pred.flatten(), True)

In [ ]:
results = pd.DataFrame({
    'y_test': y_test,
    'y_pred': y_pred.flatten()
    })

results['diff'] = y_test/y_pred.flatten()
display(results.sample(10), results['diff'].mean())